In [6]:
import csv
import re

class createBipFromFriendlist(object):
    
    def __init__(self, threshold = 5):
        """
        adjlist type: list[list] first item is follower_id, the rest are the IDs of its friends
        rtype : None
        """
        self.adjlist = None
        self.friendsCount = None
        #self.edgelist = []  #only keep  
        #self.reducedAdjlist = []
        
    def createAdjlist(self, input_file, verbose = True):
        """
        : type input_file: list[str] 
        a text file name, each row is a string 
        starting with a screen_name, and a list of id_str's  of his fiends, separated by ','
        : rtype : list[list]
         remove some rows, with 0 friends or have 'NA' as their friends, which indicates 
                    downloading unsucessfully
        """

        f = open(input_file, "r")
        adjlist = []
        failed_sn = []
        zero_sn = []
        #removed those having 0 friends, 
        #removed those having 'NA' as friends -- did not download friends list sucessfullly,
        for line in f:
            a = line.split(',')
            a =[re.sub("\n", "", x) for x in a]
            if len(a) == 1:
                zero_sn.append(a[0])
            elif len(a) ==2 and a[1] =='NA' :
                failed_sn.append(a[0]) 
            else: 
                adjlist.append(a)    #adjlist is a list[list]

        if verbose:
            print ("success>0: "+str(len(adjlist)) + ",  failed: " +str(len(failed_sn))+ ", zeros: " + str(len(zero_sn)))
        self.adjlist = adjlist

    def calculateFriendsCount(self, threshold = 1):
        """
        rtype: dict()
        key: friend IDs, values, numbers of followers in the graph
        keep the friendIds with >= self.threshold followers
        """
        friendsCount = {}
        for i in range(len(self.adjlist)):
            line = self.adjlist[i]
            for id_str in line[1:]:
                if id_str in friendsCount.keys():
                    friendsCount[id_str] = friendsCount[id_str] +1
                else:
                    friendsCount[id_str] = 1      
                    if len(friendsCount) % 5000000 ==0 :
                        print (i, len(friendsCount))
        for key in list(friendsCount.keys()):
            if friendsCount[key] < threshold:
                del friendsCount[key]
        self.friendsCount = friendsCount 
                
    def createEdgelist(self, output_file, threshold = 1):
        """
        create the edgelist from the adjlist --  write out directly to save memory&time
        type: list[list]
        list in side of [a,b] -- edge
        total length of outside list == number of edges
        """
        self.calculateFriendsCount(threshold)
        with open(output_file,'w') as out:
            csv_out=csv.writer(out)
            csv_out.writerow(['followers_sn','friends_id_str'])
            for line in self.adjlist:
                for id_str in line[1:]:
                    if id_str in self.friendsCount.keys():
                        row = (line[0],id_str)
                        csv_out.writerow(row)
            
   
                        



In [7]:
import csv
import time
start_time = time.time()
# your code

input_file = "../../data/friends_info/edgelist_Feb27/adjlist_1.txt"
output_file = "../../data/friends_info/edgelist_Feb27/edgelist_1.csv" 
BipGraph = createBipFromFriendlist()
BipGraph.createAdjlist(input_file)
BipGraph.createEdgelist(output_file, threshold = 10)
print("graph1 is done")

success>0: 56371,  failed: 4604, zeros: 53
2855 5000000
5902 10000000
9806 15000000
15979 20000000
21604 25000000
27351 30000000
33457 35000000
40049 40000000
47797 45000000
graph1 is done


In [8]:
import csv
import time
start_time = time.time()
# your code

input_file = "../../data/friends_info/edgelist_Feb27/adjlist_3.txt"
output_file = "../../data/friends_info/edgelist_Feb27/edgelist_3.csv" 
BipGraph = createBipFromFriendlist()  #clear the previous graph as well
BipGraph.createAdjlist(input_file)
BipGraph.createEdgelist(output_file, threshold = 10)
print("graph3 is done")

success>0: 105843,  failed: 19147, zeros: 47
4661 5000000
10825 10000000
14643 15000000
20136 20000000
26844 25000000
34246 30000000
43985 35000000
54296 40000000
69169 45000000
85659 50000000
graph3 is done


success>0: 105843,  failed: 19147, zeros: 47
4661 5000000
10825 10000000
14643 15000000
20136 20000000
26844 25000000
34246 30000000
43985 35000000
54296 40000000
69169 45000000
85659 50000000
graph1 is done

In [9]:
import csv
import time
start_time = time.time()
# your code

input_file = "../../data/friends_info/edgelist_Feb27/adjlist_2.txt"
output_file = "../../data/friends_info/edgelist_Feb27/edgelist_2.csv" 
BipGraph = createBipFromFriendlist()
BipGraph.createAdjlist(input_file)
BipGraph.createEdgelist(output_file, threshold = 10)
print("graph2 is done")

success>0: 162824,  failed: 28769, zeros: 151
4387 5000000
8886 10000000
13765 15000000
18195 20000000
27713 25000000
35520 30000000
44823 35000000
53225 40000000
63738 45000000
76497 50000000
93448 55000000
112132 60000000
133242 65000000
156871 70000000
graph2 is done


In [10]:
def calculateFriendsCount(adjlist, threshold = 1):
    """
    rtype: dict()
    key: friend IDs, values, numbers of followers in the graph
    keep the friendIds with >= self.threshold followers
    """
    friendsCount = {}
    for i in range(len(adjlist)):
        line = adjlist[i]
        for id_str in line[1:]:
            if id_str in friendsCount.keys():
                friendsCount[id_str] = friendsCount[id_str] +1
            else:
                friendsCount[id_str] = 1      
                if len(friendsCount) % 1000000 ==0 :
                    print (i, len(friendsCount))
                    print(i, id_str)
    for key in list(friendsCount.keys()):
        if friendsCount[key] < threshold:
            del friendsCount[key]
    return friendsCount 
        
#followerCount = calculateFriendsCount(BipGraph.adjlist, threshold = 5)


In [11]:
 def writeEdgeList_csv(self,output_file):
        with open(output_file,'w') as out:
            csv_out=csv.writer(out)
            csv_out.writerow(['follower_id_str','friends_id_str'])
            for row in self.edgelist:
                csv_out.writerow(row)

def createEdgelist(adjlist, friendsCount, output_file, threshold):
    """
    create the edgelist from the adjlist -- update self.edgelist
    type: list[list]
    list in side of [a,b] -- edge
    total length of outside list == number of edges
    """
    with open(output_file,'w') as out:
        csv_out=csv.writer(out)
        csv_out.writerow(['row_id','follower_id_str','friends_id_str'])
        for line in adjlist:
            #a = []
            #a.append(line[0])
            for id_str in line[1:]:
                if id_str in friendsCount.keys():
                    row = (line[0],id_str)
                    csv_out.writerow(row)

#createEdgelist(BipGraph.adjlist, followerCount,  "edgelist.csv", 5)

BipGraph.createEdgelist(threshold = 5)
BipGraph.writeEdgeList_csv(output_file)
elapsed_time = time.time() - start_time
print(elapsed_time)